<a href="https://colab.research.google.com/github/armandossrecife/piloto/blob/main/analise_designite_cassandra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!java --version

openjdk 11.0.20 2023-07-18
OpenJDK Runtime Environment (build 11.0.20+8-post-Ubuntu-1ubuntu122.04)
OpenJDK 64-Bit Server VM (build 11.0.20+8-post-Ubuntu-1ubuntu122.04, mixed mode, sharing)


# Funcões

In [6]:
import pandas as pd
import os
import requests
import datetime

def checa_indice_language(relacao_linguagens):
  indice=-1
  for i, each in enumerate(relacao_linguagens):
    if "Language" in each:
      indice = i
      break
  return indice

def instala_cloc():
  try:
    comando = !sudo apt-get install cloc
    print(comando)
  except Exception as e:
    print(f'Erro: {str(e)}')

def clona_repositorio(url):
  try:
    comando = f'git clone {url}'
    print(f'Clonando o repositório {url}')
    print(comando)
    os.system(comando)
    print('concluido com sucesso!')
  except Exception as e:
    print(f'Erro: {str(e)}')

def get_primeiro_commit(repositorio):
  try:
    os.environ['REPOSITORIO_ORIGINAL'] = repositorio
    # Primeiro commit
    comando = !cd $REPOSITORIO_ORIGINAL && git rev-list --all | tail -n 1
  except Exception as e:
    print(f'Erro: {str(e)}')
  return comando

def get_commit_tag(tag):
  try:
    os.environ['TAG'] = tag
    comando = !cd cassandra && git rev-list -n 1 $TAG
  except Exception as e:
    print(f'Erro: {str(e)}')
  return comando

def get_ultimo_commit(repositorio):
  try:
    os.environ['REPOSITORIO_ORIGINAL'] = repositorio
    # Primeiro commit
    comando = !cd $REPOSITORIO_ORIGINAL && git rev-list --all | head -n 1
  except Exception as e:
    print(f'Erro: {str(e)}')
  return comando

def cria_pasta_n_commit(repositorio, pasta_n_commit):
  try:
    os.environ['REPOSITORIO_ORIGINAL'] = repositorio
    os.environ['REPOSITORIO_INICIAL'] = pasta_n_commit
    # Cria uma nova pasta para referenciar apenas os dados do 1o commit
    comando = !cp -R $REPOSITORIO_ORIGINAL $REPOSITORIO_INICIAL
  except Exception as e:
    print(f'Erro: {str(e)}')
  return comando

def checkout_n_commit(pasta_n_commit, n_commit):
  try:
    os.environ['REPOSITORIO_INICIAL'] = pasta_n_commit
    os.environ['PRIMEIRO_COMMIT'] = n_commit
    # git checkout $( git rev-list --all | tail -n 1 )
    comando1 = "cd $REPOSITORIO_INICIAL && git checkout $PRIMEIRO_COMMIT"
    os.system(comando1)
    print(f'Checkout do repositorio no commit {n_commit} realizado com sucesso!')
  except Exception as e:
    print(f'Erro: {str(e)}')

def gera_df_info_repositorio(pasta_n_commit):
  try:
    os.environ['REPOSITORIO_INICIAL'] = pasta_n_commit
    # Quantidade de arquivos na pasta cassandra
    qtd_arquivos = !find $REPOSITORIO_INICIAL -type f | wc -l
    # Quantidade de commits do repositorio cassandra
    qtd_commits = !cd $REPOSITORIO_INICIAL && git log --oneline | grep -c .
    # Tamanho (MB) da pasta cassandra
    tamanho_pasta_mb = !du -hs --block-size=1M $REPOSITORIO_INICIAL | cut -f1
    # Quantidade de colaboradores (devs que commitaram pelo menos uma vez no projeto cassandra)
    qtd_colaboradores_commits = !cd $REPOSITORIO_INICIAL && git log --pretty=format:"%ae" | sort -u | grep -v "noreply.github.com" | wc -l
    # Data do primeiro commit do projeto cassandra
    data_primeiro_commit =!cd $REPOSITORIO_INICIAL && git log --reverse --pretty=format:%ad --date=short | head -1
    # Data do ultimo commit do projeto cassandra
    data_ultimo_commit = !cd $REPOSITORIO_INICIAL && git log -1 --pretty=format:%ad --date=short

    dict_info_repo = {
      'nome':pasta_n_commit,
      'qtd_arquivos':qtd_arquivos,
      'qtd_commits':qtd_commits,
      'tamanho_pasta_mb':tamanho_pasta_mb,
      'qtd_colaboradores_commits':qtd_colaboradores_commits,
      'data_primeiro_commit':data_primeiro_commit,
      'data_ultimo_commit':data_ultimo_commit,
    }

    df_info_repo = pd.DataFrame(dict_info_repo)
  except Exception as e:
    print(f'Erro: {str(e)}')
  return df_info_repo

def gera_df_dados(pasta_n_commit):
  lista_linguagens, lista_files, lista_blank, lista_comment, lista_code = list(), list(), list(), list(), list()
  l_temp = []
  vl_temp = []
  os.environ['REPOSITORIO_INICIAL'] = pasta_n_commit
  try:
    # Detalha os tipos de arquivos que aparecem no repositório do cassandra
    relacao_linguagens = !cd $REPOSITORIO_INICIAL && cloc .
    #relacao_linguagens[checa_indice_language(relacao_linguagens):]
    linguagens_encontradas = relacao_linguagens[(checa_indice_language(relacao_linguagens)+2):]
    for linha in linguagens_encontradas:
      elementos = linha.split(' ')
      novos_elementos = []
      linguagem_encontrada = []
      valores_da_linguagem = []
      for item in elementos:
        if item != '':
          novos_elementos.append(item)
      linguagem_encontrada.append(novos_elementos[:-4])
      valores_da_linguagem.append(novos_elementos[-4:])
      l_temp = l_temp + linguagem_encontrada
      vl_temp = vl_temp + valores_da_linguagem

    l_temp = l_temp[:-3]
    vl_temp = vl_temp[:-3]
    for i,each in enumerate(vl_temp):
      lista_files.append(each[0])
      lista_blank.append(each[1])
      lista_comment.append(each[2])
      lista_code.append(each[3])
  except Exception as e:
    print(f'Erro: {str(e)}')
  try:
    dict_dados_linguagens = {
    'linguagens':l_temp,
    'files':lista_files,
    'blank':lista_blank,
    'comment':lista_comment,
    'code':lista_code
    }
    df_dados_linguagens = pd.DataFrame(dict_dados_linguagens)
    df_dados_linguagens['files'] = df_dados_linguagens['files'].apply(lambda x: int(x))
    df_dados_linguagens['blank'] = df_dados_linguagens['blank'].apply(lambda x: int(x))
    df_dados_linguagens['comment'] = df_dados_linguagens['comment'].apply(lambda x: int(x))
    df_dados_linguagens['code'] = df_dados_linguagens['code'].apply(lambda x: int(x))
  except Exception as e2:
    print(f'Erro: {str(e2)}')

  return df_dados_linguagens

def gera_df_dados_repositorios(df_info_repo, df_info_repo_cf):
  df_repo = None
  try:
    l_cf = df_info_repo_cf.columns.tolist()
    c_c1 = df_info_repo.iloc[0].to_list()
    c_cf = df_info_repo_cf.iloc[0].to_list()

    dict_repo= {
        'descricao':l_cf,
        'dados_c1':c_c1,
        'dados_cf':c_cf
    }
    df_repo = pd.DataFrame(dict_repo)
  except Exception as e:
    print(f'Erro: {str(3)}')
  return df_repo

def gera_df_dados_repositorios2(df_info_repo, df_info_repo_cf):
  df_repo = None
  try:
    l_cf = df_info_repo_cf.columns.tolist()
    c_c1 = df_info_repo.iloc[0].to_list()
    c_cf = df_info_repo_cf.iloc[0].to_list()

    dict_repo= {
        'dados_cf':c_cf
    }
    df_repo = pd.DataFrame(dict_repo)
  except Exception as e:
    print(f'Erro: {str(3)}')
  return df_repo

def download_designite():
  try:
    url = 'https://www.designite-tools.com/static/download/DJC/DesigniteJava.jar'  # replace with the URL of the file you want to download
    filename = 'DesigniteJava.jar'  # replace with the desired filename to save the downloaded file
    response = requests.get(url)

    with open(filename, 'wb') as f:
        f.write(response.content)

    print(f'{filename} downloaded successfully!')
  except Excetpion as e:
    print(f'Erro: {str(e)}')

def run_designite(tag, pasta_testes):
  try:
    os.environ['TESTES'] = pasta_testes
    os.environ['CASSANDRA'] = tag + '/src/java/org/apache/cassandra'
    cria_pasta_testes = !mkdir $TESTES
    print(cria_pasta_testes)
    executa_designite = !java -jar DesigniteJava.jar -i $CASSANDRA -o $TESTES
    print(executa_designite)
  except Exception as e:
    print(f'Erro: {str(e)}')

def gera_df_design_smells(pasta_testes, lista):
  path_file = pasta_testes + '/' + 'designCodeSmells.csv'
  df_ds = pd.read_csv(path_file)
  elemento = pasta_testes,df_ds
  lista.append(elemento)
  return df_ds

In [11]:
def strings_repetidas(lista_strings):
    contador = {}
    arquivos_criticos_cd_hld = []
    # Conta a ocorrência de cada string na lista
    for string in lista_strings:
        contador[string] = contador.get(string, 0) + 1

    # Imprime as strings que se repetem pelo menos duas vezes
    for string, count in contador.items():
        if count >= 2:
            arquivos_criticos_cd_hld.append(string)
    return arquivos_criticos_cd_hld

In [ ]:
#!rm -rf cassandra_4_0_0 && rm -rf cassandra_4_1_0

# Análise Designite

In [4]:
import requests

url = 'https://www.designite-tools.com/static/download/DJC/DesigniteJava.jar'  # replace with the URL of the file you want to download
filename = 'DesigniteJava.jar'  # replace with the desired filename to save the downloaded file
response = requests.get(url)

with open(filename, 'wb') as f:
    f.write(response.content)

print(f'{filename} downloaded successfully!')

DesigniteJava.jar downloaded successfully!


In [5]:
!git clone https://github.com/apache/cassandra.git

Cloning into 'cassandra'...
remote: Enumerating objects: 393306, done.
remote: Counting objects: 100% (2555/2555), done.
remote: Compressing objects: 100% (1051/1051), done.
remote: Total 393306 (delta 1144), reused 2246 (delta 1071), pack-reused 390751
Receiving objects: 100% (393306/393306), 392.97 MiB | 23.32 MiB/s, done.
Resolving deltas: 100% (229132/229132), done.


# Análise família 3.x

## Análise da tag 3.0.0

In [7]:
# Cria uma nova pasta para referenciar apenas os dados do 1o commit
commit_tag_300 = get_commit_tag(tag='cassandra-3.0.0')
cria_pasta_n_commit(repositorio='cassandra', pasta_n_commit='cassandra_3_0_0')
checkout_n_commit('cassandra_3_0_0', commit_tag_300[0])

Checkout do repositorio no commit 96f407bce56b98cd824d18e32ee012dbb99a0286 realizado com sucesso!


In [8]:
print('Aguarde...')
t1 = datetime.datetime.now()
!java -jar DesigniteJava.jar -i cassandra_3_0_0/src/java/org/apache/cassandra -o cassandra_3_0_0
t2 = datetime.datetime.now()
print(f'Análise DesigniteJava concluída em {t2-t1}h!')

Aguarde...
Parsing the source code ...
Resolving symbols...
Extracting metrics...
Extracting code smells...
Exception while parsing literal number (during Magic Number detection). For input string: "c6a4a7935bd1e995"
Exception while parsing literal number (during Magic Number detection). For input string: "ff51afd7ed558ccd"
Exception while parsing literal number (during Magic Number detection). For input string: "c4ceb9fe1a85ec53"
Exception while parsing literal number (during Magic Number detection). For input string: "87c37b91114253d5"
Exception while parsing literal number (during Magic Number detection). For input string: "8000000000000000"
Exception while parsing literal number (during Magic Number detection). For input string: "ffff000000000000"
Exception while parsing literal number (during Magic Number detection). For input string: "ff00000000000000"
Exception while parsing literal number (during Magic Number detection). For input string: "c000000000000000"
Exception while pars

In [9]:
df_design_smells = pd.read_csv('cassandra_3_0_0/designCodeSmells.csv')
lista_classes_criticas = ['StorageService', 'DatabaseDescriptor', 'ColumnFamilyStore', 'StorageProxy', 'CompactionManager', 'SSTableReader', 'SelectStatement', 'CassandraDaemon', 'SinglePartitionReadCommand', 'NodeProbe', 'MessagingService', 'Config']
pesquisa = df_design_smells['Type Name'].str.contains('|'.join(lista_classes_criticas))
df_critical_classes = df_design_smells[pesquisa]
df_critical_classes.columns = ['ProjectName', 'PackageName', 'TypeName', 'CodeSmell']
df_critical_classes = df_critical_classes.query("CodeSmell=='Cyclic-Dependent Modularization' or CodeSmell=='Hub-like Modularization'")
df_analise_designite_critical_classes = df_critical_classes.sort_values(by='TypeName')
df_analise_designite_critical_classes.to_csv('cassandra_3_0_0/analise_designite_classes_criticas.csv', index=False)
df_design_smells.to_csv('cassandra_3_0_0/analise_designite.csv', index=False)

lista_classes_criticas = df_analise_designite_critical_classes['TypeName'].to_list()

In [12]:
classes_com_cd_hld = strings_repetidas(lista_strings=lista_classes_criticas)
print(f'{len(classes_com_cd_hld)} classes_com_cd_hld: {classes_com_cd_hld}')

7 classes_com_cd_hld: ['ColumnFamilyStore', 'CompactionManager', 'MessagingService', 'SSTableReader', 'SinglePartitionReadCommand', 'StorageProxy', 'StorageService']


In [36]:
!cd cassandra_3_0_0 && git log -n 1

commit 96f407bce56b98cd824d18e32ee012dbb99a0286 (HEAD, tag: cassandra-3.0.0)
Author: T Jake Luciani <jake@apache.org>
Date:   Fri Nov 6 14:38:34 2015 -0500

    3.0 release versions


## Análise da tag 3.11.11

In [13]:
# Cria uma nova pasta para referenciar apenas os dados do 1o commit
commit_tag_31111 = get_commit_tag(tag='cassandra-3.11.11')
cria_pasta_n_commit(repositorio='cassandra', pasta_n_commit='cassandra_3_11_11')
checkout_n_commit('cassandra_3_11_11', commit_tag_31111[0])

Checkout do repositorio no commit 4cafe2288e56e1135d65e76adbcd6c2de9306d6b realizado com sucesso!


In [14]:
print('Aguarde...')
t1 = datetime.datetime.now()
!java -jar DesigniteJava.jar -i cassandra_3_11_11/src/java/org/apache/cassandra -o cassandra_3_11_11
t2 = datetime.datetime.now()
print(f'Análise DesigniteJava concluída em {t2-t1}h!')

Aguarde...
Parsing the source code ...
Resolving symbols...
Extracting metrics...
Extracting code smells...
Exception while parsing literal number (during Magic Number detection). For input string: "c6a4a7935bd1e995"
Exception while parsing literal number (during Magic Number detection). For input string: "ff51afd7ed558ccd"
Exception while parsing literal number (during Magic Number detection). For input string: "c4ceb9fe1a85ec53"
Exception while parsing literal number (during Magic Number detection). For input string: "87c37b91114253d5"
Exception while parsing literal number (during Magic Number detection). For input string: "8000000000000000"
Exception while parsing literal number (during Magic Number detection). For input string: "ffff000000000000"
Exception while parsing literal number (during Magic Number detection). For input string: "ff00000000000000"
Exception while parsing literal number (during Magic Number detection). For input string: "c000000000000000"
Exception while pars

In [22]:
df_design_smells2 = pd.read_csv('cassandra_3_11_11/designCodeSmells.csv')
lista_classes_criticas = ['StorageService', 'DatabaseDescriptor', 'ColumnFamilyStore', 'StorageProxy', 'CompactionManager', 'SSTableReader', 'SelectStatement', 'CassandraDaemon', 'SinglePartitionReadCommand', 'NodeProbe', 'MessagingService', 'Config']
pesquisa = df_design_smells2['Type Name'].str.contains('|'.join(lista_classes_criticas))
df_critical_classes2 = df_design_smells2[pesquisa]
df_critical_classes2.columns = ['ProjectName', 'PackageName', 'TypeName', 'CodeSmell']
df_critical_classes2 = df_critical_classes2.query("CodeSmell=='Cyclic-Dependent Modularization' or CodeSmell=='Hub-like Modularization'")
df_analise_designite_critical_classes2 = df_critical_classes2.sort_values(by='TypeName')
df_analise_designite_critical_classes2.to_csv('cassandra_3_11_11/analise_designite_classes_criticas.csv', index=False)
df_design_smells2.to_csv('cassandra_3_11_11/analise_designite.csv', index=False)
lista_classes_criticas2 = df_analise_designite_critical_classes2['TypeName'].to_list()

In [23]:
classes_com_cd_hld2 = strings_repetidas(lista_strings=lista_classes_criticas2)
print(f'{len(classes_com_cd_hld2)} classes_com_cd_hld: {classes_com_cd_hld2}')

9 classes_com_cd_hld: ['ColumnFamilyStore', 'CompactionManager', 'DatabaseDescriptor', 'MessagingService', 'NodeProbe', 'SSTableReader', 'SinglePartitionReadCommand', 'StorageProxy', 'StorageService']


In [35]:
!cd cassandra_3_11_11 && git log -n 1

commit 4cafe2288e56e1135d65e76adbcd6c2de9306d6b (HEAD, tag: cassandra-3.11.11)
Author: Brandon Williams <brandonwilliams@apache.org>
Date:   Sun Jul 25 12:39:42 2021 -0500

    Prepare for release 3.11.11


# Análise família 4.x

In [ ]:
# !rm -rf cassandra_4_0_0 && !rm -rf cassandra_4_1_0

## 4.0.0

In [18]:
# Cria uma nova pasta para referenciar apenas os dados do 1o commit
commit_tag_400 = get_commit_tag(tag='cassandra-4.0.0')
cria_pasta_n_commit(repositorio='cassandra', pasta_n_commit='cassandra_4_0_0')
checkout_n_commit('cassandra_4_0_0', commit_tag_400[0])

Checkout do repositorio no commit 902b4d31772eaa84f05ffdc1e4f4b7a66d5b17e6 realizado com sucesso!


In [19]:
print('Aguarde...')
t1 = datetime.datetime.now()
!java -jar DesigniteJava.jar -i cassandra_4_0_0/src/java/org/apache/cassandra -o cassandra_4_0_0
t2 = datetime.datetime.now()
print(f'Análise DesigniteJava concluída em {t2-t1}h!')

Aguarde...
Parsing the source code ...
Resolving symbols...
Extracting metrics...
Extracting code smells...
Exception while parsing literal number (during Magic Number detection). For input string: "c6a4a7935bd1e995"
Exception while parsing literal number (during Magic Number detection). For input string: "ff51afd7ed558ccd"
Exception while parsing literal number (during Magic Number detection). For input string: "c4ceb9fe1a85ec53"
Exception while parsing literal number (during Magic Number detection). For input string: "87c37b91114253d5"
Exception while parsing literal number (during Magic Number detection). For input string: "ffffffffffffffff"
Exception while parsing literal number (during Magic Number detection). For input string: "9cb4b2f8129337db"
Exception while parsing literal number (during Magic Number detection). For input string: "a98409e882ce4d7d"
Exception while parsing literal number (during Magic Number detection). For input string: "a81e14edd9de2c7f"
Exception while pars

In [24]:
df_design_smells3 = pd.read_csv('cassandra_4_0_0/designCodeSmells.csv')
lista_classes_criticas = ['StorageService', 'DatabaseDescriptor', 'ColumnFamilyStore', 'StorageProxy', 'CompactionManager', 'SSTableReader', 'SelectStatement', 'CassandraDaemon', 'SinglePartitionReadCommand', 'NodeProbe', 'MessagingService', 'Config']
pesquisa = df_design_smells3['Type Name'].str.contains('|'.join(lista_classes_criticas))
df_critical_classes3 = df_design_smells3[pesquisa]
df_critical_classes3.columns = ['ProjectName', 'PackageName', 'TypeName', 'CodeSmell']
df_critical_classes3 = df_critical_classes3.query("CodeSmell=='Cyclic-Dependent Modularization' or CodeSmell=='Hub-like Modularization'")
df_analise_designite_critical_classes3 = df_critical_classes3.sort_values(by='TypeName')
df_analise_designite_critical_classes3.to_csv('cassandra_4_0_0/analise_designite_classes_criticas.csv', index=False)
df_design_smells3.to_csv('cassandra_4_0_0/analise_designite.csv', index=False)

lista_classes_criticas3 = df_analise_designite_critical_classes3['TypeName'].to_list()

In [ ]:
df_design_smells

,Project Name,Package Name,Type Name,Code Smell
0,cassandra,org.apache.cassandra.db,ClusteringBound,Deficient Encapsulation
1,cassandra,org.apache.cassandra.db,ClusteringBound,Broken Hierarchy
2,cassandra,org.apache.cassandra.db,ClusteringBound,Rebellious Hierarchy
3,cassandra,org.apache.cassandra.db,ClusteringBound,Cyclic-Dependent Modularization
4,cassandra,org.apache.cassandra.db,KeyspaceWriteHandler,Cyclic-Dependent Modularization
...,...,...,...,...
4145,cassandra,org.apache.cassandra.diag.store,DiagnosticEventMemoryStore,Unutilized Abstraction
4146,cassandra,org.apache.cassandra.triggers,CustomClassLoader,Unutilized Abstraction
4147,cassandra,org.apache.cassandra.triggers,ITrigger,Cyclic-Dependent Modularization
4148,cassandra,org.apache.cassandra.triggers,TriggerExecutor,Deficient Encapsulation


In [ ]:
df_analise_designite_critical_classes

,ProjectName,PackageName,TypeName,CodeSmell
2869,cassandra,org.apache.cassandra.auth,AuthConfig,Cyclic-Dependent Modularization
2626,cassandra,org.apache.cassandra.service,CassandraDaemon,Cyclic-Dependent Modularization
176,cassandra,org.apache.cassandra.db,ColumnFamilyStore,Cyclic-Dependent Modularization
178,cassandra,org.apache.cassandra.db,ColumnFamilyStore,Hub-like Modularization
906,cassandra,org.apache.cassandra.db.compaction,CompactionManager,Cyclic-Dependent Modularization
908,cassandra,org.apache.cassandra.db.compaction,CompactionManager,Hub-like Modularization
3922,cassandra,org.apache.cassandra.config,Config,Cyclic-Dependent Modularization
741,cassandra,org.apache.cassandra.db.commitlog,Configuration,Cyclic-Dependent Modularization
3920,cassandra,org.apache.cassandra.config,DatabaseDescriptor,Hub-like Modularization
3918,cassandra,org.apache.cassandra.config,DatabaseDescriptor,Cyclic-Dependent Modularization


In [25]:
classes_com_cd_hld3 = strings_repetidas(lista_strings=lista_classes_criticas3)
print(f'{len(classes_com_cd_hld3)} classes_com_cd_hld: {classes_com_cd_hld3}')

8 classes_com_cd_hld: ['ColumnFamilyStore', 'CompactionManager', 'DatabaseDescriptor', 'MessagingService', 'NodeProbe', 'SSTableReader', 'SinglePartitionReadCommand', 'StorageService']


In [34]:
!cd cassandra_4_0_0 && git log -n 1

commit 902b4d31772eaa84f05ffdc1e4f4b7a66d5b17e6 (HEAD, tag: cassandra-4.0.0)
Author: Brandon Williams <brandonwilliams@apache.org>
Date:   Thu Jul 22 12:51:39 2021 -0500

    Prepare debian changelog for 4.0.0


## 4.1.0

In [26]:
# Cria uma nova pasta para referenciar apenas os dados do 1o commit
commit_tag_410 = get_commit_tag(tag='cassandra-4.1.0')
cria_pasta_n_commit(repositorio='cassandra', pasta_n_commit='cassandra_4_1_0')
checkout_n_commit('cassandra_4_1_0', commit_tag_410[0])

Checkout do repositorio no commit f9e033f519c14596da4dc954875756a69aea4e78 realizado com sucesso!


In [27]:
print('Aguarde...')
t1 = datetime.datetime.now()
!java -jar DesigniteJava.jar -i cassandra_4_1_0/src/java/org/apache/cassandra -o cassandra_4_1_0
t2 = datetime.datetime.now()
print(f'Análise DesigniteJava concluída em {t2-t1}h!')

Aguarde...
Parsing the source code ...
Resolving symbols...
Extracting metrics...
Extracting code smells...
Exception while parsing literal number (during Magic Number detection). For input string: "8000000000000000"
Exception while parsing literal number (during Magic Number detection). For input string: "ff00000000000000"
Exception while parsing literal number (during Magic Number detection). For input string: "8000000000000000"
Exception while parsing literal number (during Magic Number detection). For input string: "ff00000000000000"
Exception while parsing literal number (during Magic Number detection). For input string: "c6a4a7935bd1e995"
Exception while parsing literal number (during Magic Number detection). For input string: "ff51afd7ed558ccd"
Exception while parsing literal number (during Magic Number detection). For input string: "c4ceb9fe1a85ec53"
Exception while parsing literal number (during Magic Number detection). For input string: "87c37b91114253d5"
Exception while pars

In [28]:
df_design_smells4 = pd.read_csv('cassandra_4_1_0/designCodeSmells.csv')
pesquisa2 = df_design_smells4['Type Name'].str.contains('|'.join(lista_classes_criticas))
df_critical_classes4 = df_design_smells4[pesquisa2]
df_critical_classes4.columns = ['ProjectName', 'PackageName', 'TypeName', 'CodeSmell']
df_critical_classes4 = df_critical_classes4.query("CodeSmell=='Cyclic-Dependent Modularization' or CodeSmell=='Hub-like Modularization'")
df_analise_designite_critical_classes4 = df_critical_classes4.sort_values(by='TypeName')
df_analise_designite_critical_classes4.to_csv('cassandra_4_1_0/analise_designite_classes_criticas.csv', index=False)
df_design_smells4.to_csv('cassandra_4_1_0/analise_designite.csv', index=False)

lista_classes_criticas4 = df_analise_designite_critical_classes4['TypeName'].to_list()

In [29]:
classes_com_cd_hld4 = strings_repetidas(lista_strings=lista_classes_criticas4)
print(f'{len(classes_com_cd_hld4)} classes_com_cd_hld: {classes_com_cd_hld4}')

8 classes_com_cd_hld: ['ColumnFamilyStore', 'CompactionManager', 'DatabaseDescriptor', 'MessagingService', 'NodeProbe', 'SSTableReader', 'SinglePartitionReadCommand', 'StorageService']


In [33]:
!cd cassandra_4_1_0 && git log -n 1

commit f9e033f519c14596da4dc954875756a69aea4e78 (HEAD, tag: cassandra-4.1.0)
Author: Mick Semb Wever <mck@apache.org>
Date:   Wed Dec 7 21:53:57 2022 +0100

    Prepare debian changelog for 4.1.0
